In [ ]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
from glob import glob
import pickle

import numpy as np
import pandas as pd
import random
import math

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import transforms
import itertools

Test for label values. <br>
(1) embeded label <br>
(2) label smoothing.


I thought if the model is continuous, zero-image has 0 label output. <br>
After that, when the image has a needle, the output moves to label one from 0.<br>
Empty -> 0.5 -> 1 (continous output)

To apply this idea, <br>
I believed it is required to give an additional dimension to the model's output. 

i.e. label = 0 -> label = (0,0) and label = 1 -> label = (1,1) <br>

- And we may consider models output is a two-dimensional vectors.<br>
(1) 2d output = (x, y).<br>
(2) Calculate distance between d1 = (x^2 + y^2) and d2 = ((x-1)^2 + (y-1)^2) <br>
(3) Taking log_softmax function on (d1, d2). <br>
(4) Apply BCE Loss <br>

I thought it may nice, <br>
since the model can classify zero-images to (0,0) output. <br>
I hope the decision boundary have eliptic shape.

Everything is same as a baseline. <br>
(add calculate distance term on (1) loss function and (2) evaluate function)

In [ ]:
def getpath(x, mode):
    return f'../input/seti-breakthrough-listen/{mode}/{x[0]}/{x}.npy'

def getpath_train(x, mode):
    return f'../input/seti-pytorch-data/train_images/{x}.npz'

test_df = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
test_df['filepath'] = test_df.id.apply(lambda x: getpath(x, 'test'))

train_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_df['filepath'] = train_df.id.apply(lambda x: getpath_train(x, 'train'))
print(train_df.head())
print(test_df.head())

In [ ]:
class setiDataset_train(Dataset):
    def __init__(self, df, transform = None): 
        self.df = df
        self.transform = transform
    
    def fileinfo(self, idx):
        return self.df.filepath.iloc[idx], self.df.target.iloc[idx]
    
    def loadfile(self, filepath):
        image = np.load(filepath)['im'].astype('float')
        return image
    
    def __getitem__(self, idx):
        filepath, target = self.fileinfo(idx)
        image = self.loadfile(filepath)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            return image , torch.tensor(target, dtype=torch.int64)
        
        image = ToTensorV2()(image=image)['image']
        return  image , torch.tensor(target, dtype=torch.int64)
    
    def __len__(self):
        return len(self.df)

In [ ]:
class setiDataset_test(Dataset):
    def __init__(self, df): 
        self.df = df
    
    def fileinfo(self, idx):
        return self.df.filepath.iloc[idx], self.df.target.iloc[idx]
    
    def loadfile(self, filepath):
        image = np.load(filepath).astype('float16')
        return image
    
    def __getitem__(self, idx):
        filepath, target = self.fileinfo(idx)
        image = self.loadfile(filepath)
        image = np.vstack([image[0], image[2], image[4]]).transpose()
        image = image.astype('float')
        
        image = ToTensorV2()(image=image)['image']
        return  image , torch.tensor(target, dtype=torch.int64)
    
    def __len__(self):
        return len(self.df)

In [ ]:
train_transform = A.Compose([
    A.VerticalFlip(p=0.1),
    A.HorizontalFlip(p=0.1),
    ToTensorV2()
])

In [ ]:
class models(nn.Module):
    def __init__(self,model_name = 'efficientnet_b0', pretrained = False):
        super(models,self).__init__()
        print(f'Model: {model_name}')
        self.backbone =  timm.create_model(model_name, pretrained=pretrained, in_chans = 1, num_classes = 2)
        
    def forward(self, x):
        x = self.backbone(x)
        
        return x

LabelSmoothing Source: https://github.com/NVIDIA/DeepLearningExamples

Just add distance values d1, d2 which represent <br>
d1 = distance between (0,0) and x, <br>
d2 = distance between (1,1) and x. 

In [ ]:
class LabelSmoothing(nn.Module):
    """NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0):
        """Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        
        # 2d eliptic distance
        d1 = torch.sum(x**2, dim =-1).view(-1,1)
        d2 = torch.sum((x-1)**2, dim = -1).view(-1,1)
        x  = torch.cat([d2,d1],dim = -1)
        
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.view(-1,1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

In [ ]:
def train(data_loader, model, optimizer, criterion, device):
    model.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        images, targets = data
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.int64)
        def closure():
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, targets)
            loss.backward()
            return loss
        optimizer.step(closure)

In [ ]:
def evaluate(data_loader, criterion, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    validate_losses = 0
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            images, targets = data

            images = images.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.int64)
            
            x = model(images)
            loss = criterion(x, targets)
            validate_losses += loss.item()

            d1 = torch.sum(x**2, dim =-1).view(-1,1)
            d2 = torch.sum((x-1)**2, dim = -1).view(-1,1)
            x  = torch.cat([d2,d1],dim = -1)
            x = torch.nn.functional.softmax(x, dim=-1)
            x = x[:,1]
        
            x = x.detach().cpu().numpy().tolist()
            targets = targets.detach().cpu().numpy().tolist()
            
            final_outputs.extend(x)
            final_targets.extend(targets)
        
    roc_auc = metrics.roc_auc_score(final_targets, final_outputs)
    
    return validate_losses/len(data_loader), roc_auc

In [ ]:
def sub_evaluate(data_loader, device):
    model.eval()
    
    final_outputs = []
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            images, targets = data

            images = images.to(device, dtype=torch.float)
            
            x = model(images)
            x = x.detach().cpu().numpy()
            
            final_outputs.append(x)
    final_outputs = np.concatenate(final_outputs)
    return final_outputs

- The output of models <br>
(x,y) coordinate. (I hope it have eliptical decision boundary.)

Using distance of two points (0,0), (1,1) to get the AUC .


In [ ]:
Batch_Size = 20
fold = 0
criterion = LabelSmoothing(smoothing=0.1)
device = torch.device("cuda")

model = models(model_name = 'efficientnet_b0', pretrained = True)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, eps=1e-5, weight_decay=1e-6)

In [ ]:
train_df['fold'] = np.random.randint(low=0, high = 3, size = len(train_df))
train_fold = train_df[train_df.fold != fold].reset_index(drop=True)
valid_fold = train_df[train_df.fold == fold].reset_index(drop=True)

train_dataset = setiDataset_train(train_fold, transform = train_transform)
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size, shuffle=True)
valid_dataset = setiDataset_train(valid_fold)
valid_loader= torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size, shuffle=False)

hold-out validation (2/3: training, 1/3 validate)

In [ ]:
for epoch in range(3):
    print(epoch)
    train(train_loader, model, optimizer, criterion, device)
    loss, roc_auc = evaluate(valid_loader, criterion, device)
    print(f'Epoch = {epoch}, Valid Loss = {loss}, AUC = {roc_auc}')

In my opinion, giving additional dimensions to the 1D label <br>
can provide continuity on the decision boundary.

I will check this by visualization of output vectors.

In [ ]:
# visualize 2-dim output
r_dataset = setiDataset_train(train_df)
r_loader= torch.utils.data.DataLoader(r_dataset, batch_size=Batch_Size, shuffle=False)

r_predict =  sub_evaluate(r_loader, device)
train_df['pred_x'] = r_predict[:,0]
train_df['pred_y'] = r_predict[:,1]


t_dataset = setiDataset_test(test_df)
t_loader= torch.utils.data.DataLoader(t_dataset, batch_size=Batch_Size, shuffle=False)

t_predict =  sub_evaluate(t_loader, device)
test_df['pred_x'] = t_predict[:,0]
test_df['pred_y'] = t_predict[:,1]

In [ ]:
import seaborn as sns

sns.scatterplot(data=train_df, x="pred_x", y="pred_y", hue="target", style="target", size=0.01 ,alpha=0.9).legend(loc=4)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.axhline(y=1, color='r', linewidth=1)
plt.axvline(x=1, color='r', linewidth=1)
plt.axhline(y=0, color='g', linewidth=1)
plt.axvline(x=0, color='g', linewidth=1)
plt.show()

In [ ]:
sns.scatterplot(data=test_df, x="pred_x", y="pred_y", size=0.01 ,alpha=0.9).legend(loc=4)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.axhline(y=1, color='r', linewidth=1)
plt.axvline(x=1, color='r', linewidth=1)
plt.axhline(y=0, color='g', linewidth=1)
plt.axvline(x=0, color='g', linewidth=1)
plt.show()

I have a feeling something is wrong with the labels 0 and 1. <br>
Maybe I can change (2,-3) and (4, -1) next time.